# New Lab Environment 

I edited the Makefile and a few more things in the environment.

My first experiment in the new environment:

    make 
    make veth_start
    make run_without_config
    make exp_zed_10
    
The **make exp_zed_10** command launches the following part of the Makefile:

    SS_PREFIX="/home/p4/p4sec/aclapolli-bmv2/targets/simple_switch"
    SS_CLI=$(SS_PREFIX)/simple_switch_CLI
    TCPREPLAY=sudo tcpreplay --preload-pcap --quiet

    PACKET_LIMIT=565248
    PACKET_RATE=3072
    PCAP_FILE=/media/p4/p4damp/datasets/zed/zed20percent-fast.pcap

    exp_zed_10:
        $(SS_CLI) < /media/p4/ddosd-p4/scripts/p4d_ddos20/control_rules_base.txt
        $(SS_CLI) < /media/p4/ddosd-p4/scripts/p4d_ddos20/control_rules_zed.txt
        echo "register_write mitigation_t 0 10" | $(SS_CLI) 
        ./$(SCRIPT_DIR)/run_capture_to_files.sh start $(OUTPUT_DIR)
        $(TCPREPLAY) --limit=$(PACKET_LIMIT) --pps=$(PACKET_RATE) --pps-multi=16 -i veth0 $(PCAP_FILE) 2>&1
        ./$(SCRIPT_DIR)/run_capture_to_files.sh stop $(OUTPUT_DIR)

       
Tcpreplay then runs the pcap through the switch. 

At last, we've obtained the same results as we had when running everything by hand.

This is excellent because it means that our experimental environment is healthy! :D 


        

## Automation

### Directory Structure

    lab/
        ddos20/
            exp_p4damp/
                if1_workload_in.pcap
                if1_workload_out.pcap
                log2m_14/
                    t_001/
                        if2_legitimate_in.pcap
                        if3_attack_in.pcap
                        if4_stats_in.pcap
                    t_002/
                    ...
                    t_020/
                log2m_16/
                    t_001/
                    t_002/
                    ...
                    t_020/
                log2m_18/
                    t_001/
                    t_002/
                    ...
                    t_020/

### Iteration

    for each m (2**14, 2**16, 2**18)
        for each j from 0.001 to 0.020 by 0.001 
            t = j*m 

### Procedure

    create directories
    add mock pcaps
    define parameters for simple_switch
        workload=if1_workload
        legitimate=log2m_xx/t_xxx/if2_legitimate
        attack=log2m_xx/t_xxx/if3_attack
        stats=log2m_xx/t_xxx/if4_stats
    start the switch and get the PID
    send control rules 
    wait for completion 
        if len(stats.pcap) > 100 * 2 ** (24-log2m)
            stop the switch
            gzip the pcaps 
            

## Building Blocks

### Get Workload Overview

Input:

- Workload Length (in packets)

- Window Length (log2)

Output:

- Lengths and offsets in packets and OWs

    - Training start/end

    - Detection 1 start/end

    - Detection 2 start/end

    - Detection 3 start/end

    - Totals
    

### Get Attack Statistics


Input

- Workload File

- Workload Length

- Window Length

- Attack Proportion

Output

- Expected legitimate/malicious packets in Detection 2

- True legitimate/malicious packets in Detection 2


### Get Traffic Characterization


Input

- Workload File

- Workload Length

- Window Length

- Sensitivity Coefficient

- Smoothing Coefficient

- Path to ee and JSON

- Path to tcad


### Run Experiment


Parameters:

- Workload File (can be long or short)

- Workload Length (in OW)

- Window Length (log2)

- Traffic Characterization

- Sensitivity Coefficient

- Smoothing Coefficient

- Mitigation Proportion Threshold

- Output PCAPs

- Script to launch, control, and terminate the switch


### Script to Control the Switch


- Get the control rules

- Get the PCAP names

- Launch the switch with a timer to start reading the workload PCAP

- Get the PID of the switch

- Submit the control rules

- Every 30 seconds:

    - Get the last changed timestamp of the statistics PCAP

    - If it is older than 30 seconds, kill the switch


### Get Experiment Statistics


Parameters:

- Path to generated output PCAPs

- Workload Length

- Window Length

Output:

- Forwarded/diverted legitimate packets

- Forwarded/diverted malicious packets

- Proportions

- Totals

# Starting BMv2

Using the 'pure' switch: 

    ./veth.sh setup 8
    ./run.sh 

Using the 'mininet' environment: 

    ./mininet.sh 
    ./control.sh
    
    

# Starting Several Wireshark Instances

    for i in {1..5..2}; do wireshark -i veth$i -k & done

In [1]:
# !putty -load "VM Lab P4"

In [2]:
# !plink -load "VM Lab P4" -batch ls